# Scanpy cheatsheet
---

## 1. Plotting

Define output folder for saving figures

In [ ]:
sc._settings.ScanpyConfig.figdir = Path(figdir)

#### Arranging scatterplots from multiple anndata objects in same line

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,6))
sc.pl.umap(adata, ax=ax[0], show=False)
sc.pl.umap(bdata, ax=ax[1], show=False)
sc.pl.umap(cdata, ax=ax[2], show=False)
plt.tight_layout(pad=3.0)
plt.show()

### Highlighting group in embedding

In [ ]:
ax = sc.pl.umap(pbmc, size=100, show=False)
sc.pl.umap(
    pbmc[pbmc.obs["bulk_labels"] == "Dendritic"],
    size=100,
    color="n_genes",
    ax=ax
)

### Plotting w zero centered color scale

Because some functions don't support vmax and vmin

In [1]:
adata_pl = adata.copy()
obscol2plot
vmin,vmax = (-3, 3)
adata_pl.obs["obscol2plot"] = [vmax if x > vmax else x for x in adata_pl.obs["obscol2plot"]]
adata_pl.obs["obscol2plot"] = [vmin if x < vmin else x for x in adata_pl.obs["obscol2plot"]]
rcParams["figure.figsize"] = [8,5]
sc.pl.scatter(adata_pl, x, y, color="obscol2plot", size=100, color_map='RdBu_r');

NameError: name 'adata' is not defined

#### PAGA plotting

- How to save position of paga graph corresponding to position of fa graph embedding 

- Number of columns can't be set when visualizing peaks in paga graph 

In [ ]:
sc.pl.paga(adata, color=feats[:int(len(feats)/2)])
sc.pl.paga(adata, color=feats[int(len(feats)/2):])


- `sc.pl.paga_compare` with continuous features works only if you have first run it coloring by clusters 

### Annotate a dim reduction

In [ ]:
## Copy pasting from https://stackoverflow.com/questions/44098362/using-mpatches-patch-for-a-custom-legend/44098900#44098900
import matplotlib.patches as mpatches 
from matplotlib.legend_handler import HandlerPatch
colors = ["g", "w"]
texts = ["Green Data Description", "RedData Description"]
class HandlerEllipse(HandlerPatch):
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        center = 0.5 * width - 0.5 * xdescent, 0.5 * height - 0.5 * ydescent
        p = mpatches.Ellipse(xy=center, width=height + xdescent,
                             height=height + ydescent)
        self.update_prop(p, orig_handle, legend)
        p.set_transform(trans)
        return [p]
                
circle_pos = t_adata_bbknn[t_adata_bbknn.obs["is_terminal"]==1].obsm["X_draw_graph_fa"][0,]
circle1 = plt.Circle(circle_pos, 200, color='r', fill=False)

fig, ax = plt.subplots(1,1, figsize=[9,9])
sc.pl.draw_graph(t_adata_bbknn, color=["palantir_pseudotime"], cmap="viridis", size=10, 
                 frameon=False,
                 ax=ax, show=False)
for i in range(sum(t_adata_bbknn.obs["is_terminal"]==1)):
#     circle_label = t_adata_bbknn[t_adata_bbknn.obs["is_terminal"]==1].obs["branch"][i]
#     circle_color = t_adata_bbknn.uns["branch_colors"][i]
    circle_color="black"
    circle_pos = t_adata_bbknn[t_adata_bbknn.obs["is_terminal"]==1].obsm["X_draw_graph_fa"][i,]
    circle = plt.Circle(circle_pos, 300, linewidth=3, color=circle_color, fill=False)
    circle_legend = plt.Line2D([circle_pos[0]], [circle_pos[1]], color="white", marker='o', markerfacecolor="red")
    ax.add_artist(circle);
    ax.text(circle_pos[0], circle_pos[1]-1000, i+1, color=circle_color, fontsize=15)
# ax.legend([circle], labels=["cia"], numpoints=1, loc=1)
ax.legend([circle],["Terminal states"],bbox_to_anchor=(0.2, 0.95), loc='center', ncol=2, 
          handler_map={mpatches.Circle: HandlerEllipse()}, 
          frameon=False)


### Changing the colorbar ticks

Inspiration from here https://github.com/theislab/scanpy/issues/337

In [ ]:
ax = sc.pl.draw_graph(t_adata_bbknn, color=["branch_prob_neglect_DP"], size=30,  
                 frameon=False, cmap="PRGn",
                    vmin=0, vmax=1,
                 title=["branch probability"],
                 show=False
                )

fig = plt.gcf()
cbar_ax = fig.axes[-1]
cbar_ax.set_yticks(np.array([0,1]));
cbar_ax.set_yticklabels(labels=["unconv_SP", "", "", "", "","neglect_DP"]);
fig.savefig(figdir + "draw_graph_branchprobs.png")

---
## 2. AnnData

Delete a dimensionality reduction matrix (slot from `adata.obsm`)

In [ ]:
adata.obsm.__delitem__("X_pca")

In [2]:
import scanpy as sc
adata = sc.read_h5ad("/nfs/team205/ed6/data/thymus_multimap/thymusATAC_cellatacPeaks_tcellsBBKNN_20200911.h5ad")

AnnData object with n_obs × n_vars = 19607 × 68108
    obs: 'nCount_peaks', 'nFeature_peaks', 'total', 'duplicate', 'chimeric', 'unmapped', 'lowmapq', 'mitochondrial', 'passed_filters', 'cell_id', 'TSS_fragments', 'DNase_sensitive_region_fragments', 'enhancer_region_fragments', 'promoter_region_fragments', 'on_target_fragments', 'blacklist_region_fragments', 'peak_region_fragments', 'peak_region_cutsites', 'peaks_snn_res.1.2', 'seurat_clusters', 'sample_id', 'nCount_win', 'nFeature_win', 'Age', 'annotation_1', 'leiden', 'leiden_names', 'is_root', 'CD34p_CD4m_CD1m', 'CD34p_CD4m_CD1p', 'CD34p_CD4p', 'DP_CD3m', 'DP_CD3p', 'ISP_CD28p', 'SP_CD4p', 'SP_CD8p', 'gd_CD1m', 'gd_CD1p', 'CD34+CD1-', 'CD34+CD1+', 'CD34+CD4+', 'ISP CD28+', 'DP CD3-', 'DP CD3+', 'SP CD4+', 'SP CD8+', 'γδ CD1+', 'γδ CD1-', 'palantir_pseudotime', 'palantir_entropy', 'differentiation potential', 'is_terminal', 'terminal_state', 'branch_prob_mature_T', 'branch_prob_neglect_DP'
    var: 'tot_count', 'tot_cells', 'frac_cel